In [60]:
import numpy as np
import pandas as pd
import re
from ast import literal_eval

In [61]:
df = pd.read_csv(r'.\small_eval.csv', index_col=False)
# df = df.head(10)
print(df['recommended'].iloc[2])

['The Great Gatsby', '1984', 'The Hunger Games', 'The Catcher in the Rye', 'Animal Farm', "Harry Potter and the Sorcerer's Stone", 'Lord of the Flies', 'Of Mice and Men', 'Twilight', 'The Fault in Our Stars', 'Pride and Prejudice', 'The Diary of a Young Girl', 'Divergent', 'Romeo and Juliet', 'The Giver', 'Fahrenheit 451', 'The Kite Runner', 'The Help', 'The Hobbit', 'The Book Thief']


In [62]:
for i in range(30):
    x = str(df['recommended'].iloc[i])[1:-1]
    thelist = x.split(',')
    result = []
    for y in thelist:
        result.append(y.strip())

        
    print(result)
    print(len(result))

["'Divergent'", "'Twilight'", '"Harry Potter and the Sorcerer\'s Stone"', "'The Fault in Our Stars'", "'To Kill a Mockingbird'", "'The Lightning Thief'", "'The Maze Runner'", "'The Great Gatsby'", "'The Giver'", "'The Hobbit'", "'Harry Potter and the Prisoner of Azkaban'", "'Fifty Shades of Grey'", "'Pride and Prejudice'", "'The Lion", "the Witch and the Wardrobe'", "'Paper Towns'", "'The Diary of a Young Girl'", "'The Selection'", "'1984'", "'The Help'", "'Insurgent'"]
21
["'Harry Potter and the Cursed Child: Parts One and Two'", "'Mockingjay'", "'Catching Fire'", "'Breaking Dawn'", "'The Sea of Monsters'", "'New Moon'", "'The Hunger Games'", "'The Lightning Thief'", '"The Titan\'s Curse"', "'Eclipse'", "'The Last Olympian'", "'The Battle of the Labyrinth'", "'The Chronicles of Narnia'", "'Fantastic Beasts and Where to Find Them'", "'Insurgent'", "'The Da Vinci Code'", "'Harry Potter and the Chamber of Secrets: Sheet Music for Flute with CD'", "'The Two Towers'", "'Allegiant'", "'Quid

In [63]:
df['recommended'] = df['recommended'].apply(literal_eval)
print(df['recommended'])
for i in range(30):
    print(len(df['recommended'].iloc[i]))


0     [Divergent, Twilight, Harry Potter and the Sor...
1     [Harry Potter and the Cursed Child: Parts One ...
2     [The Great Gatsby, 1984, The Hunger Games, The...
3     [Jane Eyre, The Great Gatsby, Twilight, Harry ...
4     [The Hunger Games, Harry Potter and the Sorcer...
5     [All the Light We Cannot See, To Kill a Mockin...
6     [The Great Gatsby, To Kill a Mockingbird, Lord...
7     [J.R.R. Tolkien 4-Book Boxed Set: The Hobbit a...
8     [The Chronicles of Narnia, The Belgariad Boxed...
9     [Scarlett, Memoirs of a Geisha, Girl with a Pe...
10    [The Hunger Games, Twilight, Divergent, Harry ...
11    [Ender's Game, The Hobbit, Dune, The Martian, ...
12    [Where the Wild Things Are, Green Eggs and Ham...
13    [Jane Eyre, Pride and Prejudice, Sense and Sen...
14    [Harry Potter and the Prisoner of Azkaban, The...
15    [The Help, The Kite Runner, Water for Elephant...
16    [Wuthering Heights, Frankenstein: The 1818 Tex...
17    [Alice in Wonderland, The Chronicles of Na

In [64]:
def precisionatk(r,k=20): # precision is relevant&retrieved/retrieved
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    totalp=0.0
    count = 0.0
    for i in range(len(r)):
        if r[i] == 1:
            totalp+=precisionatk(r,i+1)
            count+=1.0
    if count==0.0: return 0.0
    avg_p = totalp/count
    return avg_p

def mean_average_precision(rs):
    totalp = 0.0
    for r in rs:
        totalp+= average_precision(r)
    m_avg_p = totalp/len(rs)

    return m_avg_p

In [65]:
def dcg_at_k(r, k, method=0):
    dcg = 0.0
    for i in range(k):
        if i==0:
            dcg += r[i]
        elif i>=len(r):
            return dcg
        elif r[i] != 0:
            dcg += r[i]/log2(i+1)

    return dcg

def ndcg_at_k(r, k, method=0):
    if len(r) ==1:
        return r[0]
    sorted_r = sorted(r,reverse=True)
    # print(sorted_r)
    ndcg = []
    for i in range(len(sorted_r)):
        dcg = dcg_at_k(r,i+1)
        idcg= dcg_at_k(sorted_r,i+1)
        ndcg.append(dcg/idcg)
    # print(ndcg)
    
    return ndcg[k-1]